In [0]:
!apt -y install libcusparse8.0 libnvrtc8.0 libnvtoolsext1
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!pip install 'cupy-cuda80==4.0.0b4' 'chainer==4.0.0b4'

# cifar10データ読み込み

In [0]:
%matplotlib inline
import os
import cv2
import matplotlib
import matplotlib.pyplot as plt
#dataset
from chainer.datasets import get_cifar10
train, test = get_cifar10(withlabel=True)

print("==========")
print("学習データ")
check(train[0][0])
print("ラベル", train[0][1])

In [0]:
print("データ数：", len(train))

# 手動で探索的にモデルを変更する

In [0]:
class Base(object):
        def get_class_name(self):
                return self.__class__.__name__
b = Base()
b.get_class_name()

In [0]:
# -*- coding: utf-8 -*-
import chainer
from chainer import functions as F
from chainer import links as L
from chainer import Variable
from chainer import optimizers
from chainer import training, datasets
from chainer.training import extensions
import numpy as np

class Runner(object):
        def __init__(self, epoch, model, opt):
              self.model = model
              self.batch_size = 128
              self.num_classes = 100
              self.epochs = epoch
              self.chanel = 3
              self.val_interval = 100, 'iteration'
              self.log_interval = 100, 'iteration'
              chainer.cuda.get_device_from_id(0).use()
              self.model = self.model(self.chanel, self.num_classes)
              self.model.to_gpu()
              self.rate_shift = opt["rate_shift"]
              if opt["name"] == "adam":
#                       self.optimizer = optimizers.Adam(alpha=0.0002, beta1=0.5)
#                       self.optimizer = optimizers.Adam(beta1=0.9, beta2=0.999)
                          self.optimizer = optimizers.Adam()
              else:
                      self.optimizer = optimizers.SGD(lr=0.01)
              self.optimizer.setup(self.model)

        def set_dataset(self, train, test):
              self.train_iter = chainer.iterators.SerialIterator(train, self.batch_size)
              self.test_iter = chainer.iterators.SerialIterator(test, self.batch_size, repeat=False, shuffle=False)
        def train(self):
              updater = training.StandardUpdater(self.train_iter, self.optimizer, device=0)
              trainer = training.Trainer(updater, (self.epochs, 'epoch'), out=self.model.get_class_name()+"_result")
              trainer.extend(extensions.LogReport(trigger=self.log_interval))
              trainer.extend(extensions.observe_lr(), trigger=self.log_interval)
              trainer.extend(extensions.Evaluator(self.test_iter, self.model, device=0), trigger=self.val_interval)
              trainer.extend(extensions.PrintReport([
                  'epoch', 'iteration', 'main/loss', 'validation/main/loss',
                  'main/accuracy', 'validation/main/accuracy', 'lr'
              ]), trigger=self.log_interval)
              trainer.extend(extensions.PlotReport(['main/loss', 'validation/main/loss'], x_key='epoch', file_name='loss.png'))
              trainer.extend(extensions.PlotReport(['main/accuracy', 'validation/main/accuracy'], x_key='epoch', file_name='accuracy.png'))
              trainer.extend(extensions.ProgressBar(update_interval=100))
#               if self.rate_shift == True:
#                       trainer.extend(extensions.ExponentialShift('lr', 0.005), trigger=(3, 'epoch'))
#                       trainer.extend(extensions.ExponentialShift('lr', 0.0001), trigger=(6, 'epoch'))
              trainer.run()

        def predict(self, image):
                xp = cuda.cupy
                return self.model.predict(xp.asarray(x))
                
#     for x_train_ in train_iter:
#       print(x_train_[0][0].shape)
#       x = x_train_[0][0].reshape(1, 28, 28, 1).transpose(3, 0, 1, 2)
#       print(mlp_model.predict(xp.asarray(x)))
#       break
        

In [0]:
opt = {}
opt["name"] = "adam"
opt["rate_shift"] = False

In [0]:
class Cnn1(chainer.Chain, Base):
    def __init__(self, n_units, n_out):
        super(Cnn1, self).__init__()
        with self.init_scope():
            #Convolution2D(self, in_channels, out_channels, ksize=None, stride=1, pad=0, nobias=False, initialW=None, initial_bias=None)
            self.conv1 = L.Convolution2D(None, 32, ksize=3)
            self.conv2 = L.Convolution2D(None, 32*2, ksize=3)
            self.conv3 = L.Convolution2D(None, 32*3, ksize=3)
            self.loss1_fc1 = L.Linear(None, n_out)
            
    def __call__(self, input_cnn, t):
        cnn1 = F.relu(self.conv1(input_cnn))
        cnn2 = F.relu(self.conv2(cnn1))
        cnn3 = F.relu(self.conv3(cnn2))
        fc1 = self.loss1_fc1(cnn3)
        loss = F.softmax_cross_entropy(fc1, t)
        #accuracy
        accuracy = F.accuracy(fc1, t)
        chainer.report({
            'loss': loss,
            'accuracy': accuracy
        }, self)
        return loss
      
    def predict(self, input_cnn):
        cnn1 = F.relu(self.conv1(input_cnn))
        cnn2 = F.relu(self.conv2(cnn1))
        cnn3 = F.relu(self.conv3(cnn2))
        fc1 = self.loss1_fc1(cnn3)
        return F.softmax(fc1)

In [0]:
runner = Runner(10, Cnn1, opt)
runner.set_dataset(train, test)
runner.train()

In [0]:
from IPython.display import Image
Image(filename='Cnn1_result/loss.png')

In [0]:
from IPython.display import Image
Image(filename='Cnn1_result/accuracy.png')

In [0]:
class Cnn2(chainer.Chain, Base):
    def __init__(self, n_units, n_out):
      super(Cnn2, self).__init__()
      with self.init_scope():
        self.conv1 = L.Convolution2D(None, 32, ksize=3)
        self.conv2 = L.Convolution2D(None, 32*2, ksize=3)
        self.conv3 = L.Convolution2D(None, 32*3, ksize=3)
        self.fc1 = L.Linear(None, n_out)
        

    def __call__(self, input_cnn, t):
        h = self.conv1(input_cnn)
        h = F.max_pooling_2d(h, ksize=2, stride=2)
        h = self.conv2(h)
        h = F.max_pooling_2d(h, ksize=2, stride=2)
        h = self.conv3(h)
        h = F.max_pooling_2d(h, ksize=2, stride=2)
        h = self.fc1(h)
        loss = F.softmax_cross_entropy(h, t)
        #accuracy
        accuracy = F.accuracy(h, t)
        chainer.report({
            'loss': loss,
            'accuracy': accuracy
        }, self)
        return loss
      
    def predict(self, input_cnn):
        h = self.conv1(input_cnn)
        h = F.max_pooling_2d(h, ksize=2, stride=2)
        h = self.conv2(h)
        h = F.max_pooling_2d(h, ksize=2, stride=2)
        h = self.conv3(h)
        h = F.max_pooling_2d(h, ksize=2, stride=2)
        h = self.fc1(h)
        return F.softmax(h)

In [0]:
runner = Runner(10, Cnn2, opt)
runner.set_dataset(train, test)
runner.train()

In [0]:
from IPython.display import Image
Image(filename='Cnn2_result/loss.png')

In [0]:
from IPython.display import Image
Image(filename='Cnn2_result/accuracy.png')

In [0]:
runner = Runner(20, Cnn2, opt)
runner.set_dataset(train, test)
runner.train()

In [0]:
from IPython.display import Image
Image(filename='Cnn2_result/loss.png')

In [0]:
from IPython.display import Image
Image(filename='Cnn2_result/accuracy.png')

In [0]:
class Cnn3(chainer.Chain, Base):
    def __init__(self, n_units, n_out):
      super(Cnn3, self).__init__()
      with self.init_scope():
        self.conv1_1 = L.Convolution2D(None, 32, ksize=3, stride=1)
        self.conv1_2 = L.Convolution2D(None, 32, ksize=3, stride=1)
        self.conv2_1 = L.Convolution2D(None, 32*2, ksize=3, stride=1)
        self.conv2_2 = L.Convolution2D(None, 32*2, ksize=3, stride=1)
        self.fc1 = L.Linear(None, 512)
        self.fc2 = L.Linear(None, n_out)
        

    def __call__(self, input_cnn, t):
        h = F.relu(self.conv1_1(input_cnn))
        h = F.relu(self.conv1_2(h))
        h = F.max_pooling_2d(h, ksize=2)
        h = F.dropout(h, ratio=0.25)
        h = F.relu(self.conv2_1(h))
        h = F.relu(self.conv2_2(h))
        h = F.max_pooling_2d(h, ksize=2)
        h = F.dropout(h, ratio=0.25)
        h = F.relu(self.fc1(h))
        h = F.dropout(h, ratio=0.5)
        h = self.fc2(h)
        loss = F.softmax_cross_entropy(h, t)
        #accuracy
        accuracy = F.accuracy(h, t)
        chainer.report({
            'loss': loss,
            'accuracy': accuracy
        }, self)
        return loss
      
    def predict(self, input_cnn):
        h = F.relu(self.conv1_1(input_cnn))
        h = F.relu(self.conv1_2(h))
        h = F.max_pooling_2d(h, ksize=2, stride=2)
        h = F.dropout(h, ratio=0.25)
        h = F.relu(self.conv2_1(h))
        h = F.relu(self.conv2_2(h))
        h = F.max_pooling_2d(h, ksize=2, stride=2)
        h = F.dropout(h, ratio=0.25)
        h = self.fc1(h)
        h = self.fc2(h)
        return F.softmax(h)

In [0]:
runner = Runner(20, Cnn3, opt)
runner.set_dataset(train, test)
runner.train()

In [0]:
from IPython.display import Image
Image(filename='Cnn3_result/loss.png')

In [0]:
from IPython.display import Image
Image(filename='Cnn2_result/accuracy.png')

In [0]:
class Cnn4(chainer.Chain, Base):
    def __init__(self, n_units, n_out):
      super(Cnn4, self).__init__()
      with self.init_scope():
        self.conv1_1 = L.Convolution2D(None, 32, ksize=3, pad=1)
        self.conv1_2 = L.Convolution2D(None, 32, ksize=3, pad=1)
        self.conv2_1 = L.Convolution2D(None, 32*2, ksize=3, pad=1)
        self.conv2_2 = L.Convolution2D(None, 32*2, ksize=3, pad=1)
        self.fc1 = L.Linear(None, 512)
        self.fc2 = L.Linear(None, n_out)
        

    def __call__(self, input_cnn, t):
        h = F.relu(self.conv1_1(input_cnn))
        h = F.relu(self.conv1_2(h))
        h = F.max_pooling_2d(h, ksize=2)
        h = F.dropout(h, ratio=0.25)
        h = F.relu(self.conv2_1(h))
        h = F.relu(self.conv2_2(h))
        h = F.max_pooling_2d(h, ksize=2)
        h = F.dropout(h, ratio=0.25)
        h = F.relu(self.fc1(h))
        h = F.dropout(h, ratio=0.5)
        h = self.fc2(h)
        loss = F.softmax_cross_entropy(h, t)
        #accuracy
        accuracy = F.accuracy(h, t)
        chainer.report({
            'loss': loss,
            'accuracy': accuracy
        }, self)
        return loss
      
    def predict(self, input_cnn):
        h = F.relu(self.conv1_1(input_cnn))
        h = F.relu(self.conv1_2(h))
        h = F.max_pooling_2d(h, ksize=2, stride=2)
        h = F.dropout(h, ratio=0.25)
        h = F.relu(self.conv2_1(h))
        h = F.relu(self.conv2_2(h))
        h = F.max_pooling_2d(h, ksize=2, stride=2)
        h = F.dropout(h, ratio=0.25)
        h = self.fc1(h)
        h = self.fc2(h)
        return F.softmax(h)

In [0]:
runner = Runner(40, Cnn4, opt)
runner.set_dataset(train, test)
runner.train()

In [0]:
# class VGG(chainer.Chain, Base):

#     def __init__(self, n_units, n_out):
#         super(VGG, self).__init__()
#         with self.init_scope():
#             self.conv1_1 = L.Convolution2D(None, 64, 3, pad=1)
#             self.bn1_1 = L.BatchNormalization(64)
#             self.conv1_2 = L.Convolution2D(64, 64, 3, pad=1)
#             self.bn1_2 = L.BatchNormalization(64)

#             self.conv2_1 = L.Convolution2D(64, 128, 3, pad=1)
#             self.bn2_1 = L.BatchNormalization(128)
#             self.conv2_2 = L.Convolution2D(128, 128, 3, pad=1)
#             self.bn2_2 = L.BatchNormalization(128)

#             self.conv3_1 = L.Convolution2D(128, 256, 3, pad=1)
#             self.bn3_1 = L.BatchNormalization(256)
#             self.conv3_2 = L.Convolution2D(256, 256, 3, pad=1)
#             self.bn3_2 = L.BatchNormalization(256)
#             self.conv3_3 = L.Convolution2D(256, 256, 3, pad=1)
#             self.bn3_3 = L.BatchNormalization(256)
#             self.conv3_4 = L.Convolution2D(256, 256, 3, pad=1)
#             self.bn3_4 = L.BatchNormalization(256)

#             self.fc4 = L.Linear(None, 1024)
#             self.fc5 = L.Linear(1024, 1024)
#             self.fc6 = L.Linear(1024, n_out)

#     def __call__(self, x, t):
#         h = F.relu(self.bn1_1(self.conv1_1(x)))
#         h = F.relu(self.bn1_2(self.conv1_2(h)))
#         h = F.max_pooling_2d(h, 2, 2)
#         h = F.dropout(h, ratio=0.25)

#         h = F.relu(self.bn2_1(self.conv2_1(h)))
#         h = F.relu(self.bn2_2(self.conv2_2(h)))
#         h = F.max_pooling_2d(h, 2, 2)
#         h = F.dropout(h, ratio=0.25)

#         h = F.relu(self.bn3_1(self.conv3_1(h)))
#         h = F.relu(self.bn3_2(self.conv3_2(h)))
#         h = F.relu(self.bn3_3(self.conv3_3(h)))
#         h = F.relu(self.bn3_4(self.conv3_4(h)))
#         h = F.max_pooling_2d(h, 2, 2)
#         h = F.dropout(h, ratio=0.25)

#         h = F.dropout(F.relu(self.fc4(h)), ratio=0.5)
#         h = F.dropout(F.relu(self.fc5(h)), ratio=0.5)
#         h = self.fc6(h)
        
#         loss = F.softmax_cross_entropy(h, t)
#         #accuracy
#         accuracy = F.accuracy(h, t)
#         chainer.report({
#             'loss': loss,
#             'accuracy': accuracy
#         }, self)
#         return loss
      

In [0]:
runner = Runner(10, VGG, opt)
runner.set_dataset(train, test)
runner.train()